In [ ]:
!pip install flaml

In [ ]:
import pandas as pd
train=pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')
test.head()

In [ ]:
train.info()

In [ ]:
train.nunique()

In [ ]:
train=train.drop(['id'],axis=1)
train.head()

In [ ]:
test=test.drop(['id'],axis=1)
test.head()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
round(train.isnull().sum()*100/len(train),2)

In [ ]:
train[train.isna().any(axis=1)].head()

In [ ]:
counter=0
for i in test.select_dtypes(include=['object']).columns.tolist():
    if (len(list(set(train[i].unique().tolist())^set(test[i].unique().tolist())))!=0):
        print(i ,'need to be worked on')
        counter+=1
    else:
        continue
if(counter==0):
    print('No work needed')

In [ ]:
train[test.select_dtypes(include=['object','bool']).columns.tolist()].head()

In [ ]:
gender=train['gender'].value_counts().to_frame()
gender.reset_index()

In [ ]:
marital_status=train['marital_status'].value_counts().to_frame()
marital_status.reset_index()

In [ ]:
education_level=train['education_level'].value_counts().to_frame()
education_level.reset_index()

In [ ]:
employment_status=train['employment_status'].value_counts().to_frame()
employment_status.reset_index()

In [ ]:
loan_purpose=train['loan_purpose'].value_counts().to_frame()
loan_purpose.reset_index()

In [ ]:
grade_subgrade=train['grade_subgrade'].value_counts().to_frame()
grade_subgrade.reset_index()

In [ ]:
train.duplicated().value_counts()

In [ ]:
round(train['loan_paid_back'].value_counts()*100/len(train),2)

In [ ]:
train['loan_paid_back'].value_counts()

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
y = train.pop('loan_paid_back')
X = train

In [ ]:
automl.fit(X, y, task="classification",metric='roc_auc',time_budget=3600*9,eval_method='cv')

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best roc_auc  on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
y_pred = automl.predict_proba(test)[:,1]
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['loan_paid_back'])
df.head()

In [ ]:
sol=pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
sol.head()

In [ ]:
sol['loan_paid_back']=df['loan_paid_back']
sol.head()

In [ ]:
sol.to_csv('./roc_auc.csv',index=False)